In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import statistics

In [3]:
import devanalyst.simulation.statics as S_
from devanalyst.simulation.simulationModels import Distribution

importing Jupyter notebook from c:\alex\code\labs\devanalyst\devanalyst\simulation\statics.ipynb
importing Jupyter notebook from c:\alex\code\labs\devanalyst\devanalyst\simulation\simulationModels.ipynb
importing Jupyter notebook from c:\alex\code\labs\devanalyst\devanalyst\simulation\GenerateTimecards.ipynb


<h1>Timecard Visualizations</h1>

<h2>Visualize runReleaseCycle's log</h2>

In [18]:
def _getTarget(sprint, teamLog, targetPhase):
    target = []
    for item in teamLog[sprint]['unplanned_' + targetPhase]:
        target.append(item.estimate)
    return target

In [22]:
def _renderRCLHelper(sprint, teamLog, target, sprintDuration, figure, snapshot, ax, color):

    data = []
    for item in teamLog[sprint][snapshot]:
        data.append(item.estimate)
            
    sns.distplot(data, bins=sprintDuration, color=color, ax=ax)
    ax.set_title(snapshot)
    if len(data)==0:
        mean = 0
    else:
        mean = statistics.mean(data)
        
    dataDist = Distribution.sample_to_dist(data, sprintDuration)
    targetDist = Distribution.sample_to_dist(target, sprintDuration)
    distance = Distribution.measureDistributionDistance(dataDist, targetDist)
    
    ax.set_xlabel('size=' + str(len(data)) + '; mean=' + str(round(mean, 2)) + '; distance=' + str(round(distance,2)))

In [21]:
# -release_log: a ReleaseLog instance.
def renderReleaseCycleLog(teamId, release_log, sprintDuration, first, last):
    
    sprints = list(release_log.log[teamId].keys())
    log = release_log.log # release_log is a ReleaseLog instance, and 'log' is a dict
    
    phases = ['T0','T1','T2','T3']
    
    for sprint in sprints:
        if sprint < first or sprint > last: # Only run for the chosen sprints
            continue
        target = _getTarget(sprint, log[teamId], 'T0')
            
        fig, axs = plt.subplots(len(phases), 3)   

        fig.suptitle('--------------------------------------------- SPRINT '+ str(sprint) + 
                     ' ---------------------------------------------', fontsize=16, y=1.4, x=1)

        for idx in range(len(phases)):
            phase = phases[idx]
            _renderRCLHelper(sprint, log[teamId], target, sprintDuration, fig, \
                            'unplanned_' + phase, axs[idx][0], 'g')
            _renderRCLHelper(sprint, log[teamId], target, sprintDuration, fig, \
                            'planned_' + phase + '_CURRENT_SPRINT', axs[idx][1], 'b')
            _renderRCLHelper(sprint, log[teamId], target, sprintDuration, fig, \
                            'planned_' + phase + '_NEXT_SPRINT', axs[idx][2], 'b')
                             
        plt.subplots_adjust(top=1.2, right=2.3, wspace=0.3, hspace=1.5)